In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
base_dir = 'runtime_data'
files_path = {file_name.split('_')[0]: os.path.join(base_dir, file_name) for file_name in os.listdir(base_dir)}
files_path = {k: v for k, v in files_path.items() if k not in ['roofline', 'models', 'batch']}
df_dict = {model_name: pd.read_csv(model_path) for model_name, model_path in files_path.items()}
df_dict.keys()

dict_keys(['lstm', 'yolov8n', 'mobnet', 'resnet50', 'bert-large'])

In [3]:
filtered_df_dict = {model_name: df[(df['cores'] == 12) & 
                                   (df['cpu'] == 2201600) & 
                                   (df['gpu'] == 1300500000) & 
                                   (df['mem'] == 3199000000)] 
                    for model_name, df in df_dict.items()}
filtered_df_dict

{'lstm':       cores      cpu         gpu         mem  observed_time    bs  \
 440      12  2201600  1300500000  3199000000       1.232528   1.0   
 881      12  2201600  1300500000  3199000000       1.522864   4.0   
 1322     12  2201600  1300500000  3199000000       1.883888  16.0   
 1763     12  2201600  1300500000  3199000000       2.411056  32.0   
 2204     12  2201600  1300500000  3199000000       3.364736  64.0   
 
       observed_power  
 440           29.474  
 881           30.487  
 1322          29.488  
 1763          35.816  
 2204          41.436  ,
 'yolov8n':       cores      cpu         gpu         mem  observed_time    bs  \
 440      12  2201600  1300500000  3199000000      17.943621   1.0   
 881      12  2201600  1300500000  3199000000      30.361891   4.0   
 1322     12  2201600  1300500000  3199000000     107.458115  16.0   
 1763     12  2201600  1300500000  3199000000     212.697983  32.0   
 2204     12  2201600  1300500000  3199000000     431.465030  64

In [4]:
models_metrics = {model_name: {} for model_name in files_path.keys()}  # contains ai and performance for each batch size
models_metrics

{'lstm': {}, 'yolov8n': {}, 'mobnet': {}, 'resnet50': {}, 'bert-large': {}}

In [5]:
const = {'mobnet': {'flops': {1: 449137560,
   2: 898275120,
   4: 1796550240,
   8: 3593100480,
   16: 7186200960,
   32: 14372401920,
   64: 28744803840,
   128: 57489607680,
   256: 114979215360,
   512: 229958430720,
   1024: 459916861440},
  'param_mem': {1: 21805088,
   2: 21805088,
   4: 21805088,
   8: 21805088,
   16: 21805088,
   32: 21805088,
   64: 21805088,
   128: 21805088,
   256: 21805088,
   512: 21805088,
   1024: 21805088},
  'mem_acc': {1: 102976672,
   2: 184148256,
   4: 346491424,
   8: 671177760,
   16: 1320550432,
   32: 2619295776,
   64: 5216786464,
   128: 10411767840,
   256: 20801730592,
   512: 41581656096,
   1024: 83141507104},
  'param_frac': {1: 0.21174784129749308,
   2: 0.11841050506609196,
   4: 0.06293110446508483,
   8: 0.032487798761389235,
   16: 0.016512120606386654,
   32: 0.008324790273704469,
   64: 0.004179793087271743,
   128: 0.0020942733582887878,
   256: 0.001048234323753134,
   512: 0.0005243920047257947,
   1024: 0.0002622647671364011},
  'ai': {1: 4.361546661752674,
   2: 4.877999604840135,
   4: 5.184977507552972,
   8: 5.353426013400682,
   16: 5.441822429391322,
   32: 5.487124459822746,
   64: 5.510059504708913,
   128: 5.521599075532211,
   256: 5.5273870052051866,
   512: 5.530285522757742,
   1024: 5.531735921802565}},
 'resnet50': {'flops': {1: 8206316544,
   2: 16412633088,
   4: 32825266176,
   8: 65650532352,
   16: 131301064704,
   32: 262602129408,
   64: 525204258816,
   128: 1050408517632,
   256: 2100817035264,
   512: 4201634070528,
   1024: 8403268141056},
  'param_mem': {1: 102011648,
   2: 102011648,
   4: 102011648,
   8: 102011648,
   16: 102011648,
   32: 102011648,
   64: 102011648,
   128: 102011648,
   256: 102011648,
   512: 102011648,
   1024: 102011648},
  'mem_acc': {1: 336671392,
   2: 571331136,
   4: 1040650624,
   8: 1979289600,
   16: 3856567552,
   32: 7611123456,
   64: 15120235264,
   128: 30138458880,
   256: 60174906112,
   512: 120247800576,
   1024: 240393589504},
  'param_frac': {1: 0.30300064224049067,
   2: 0.1785508290589645,
   4: 0.09802679751239932,
   8: 0.05153952610067774,
   16: 0.026451409608292013,
   32: 0.013402968509147252,
   64: 0.006746697139222503,
   128: 0.0033847665670687405,
   256: 0.0016952523001885827,
   512: 0.0008483452296952887,
   1024: 0.0004243526136053748},
  'ai': {1: 24.374855538661272,
   2: 28.727006203281736,
   4: 31.54302262350827,
   8: 33.16873506130685,
   16: 34.04609485860239,
   32: 34.50241359585168,
   64: 34.73519093095508,
   128: 34.85276144391893,
   256: 34.91184566792465,
   512: 34.941462965656896,
   1024: 34.95629046678957}},
 'yolov8n': {'flops': {1: 8839855200,
   2: 17679710400,
   4: 35359420800,
   8: 70718841600,
   16: 141437683200,
   32: 282875366400,
   64: 565750732800,
   128: 1131501465600,
   256: 2263002931200,
   512: 4526005862400,
   1024: 9052011724800},
  'param_mem': {1: 12584704,
   2: 12584704,
   4: 12584704,
   8: 12584704,
   16: 12584704,
   32: 12584704,
   64: 12584704,
   128: 12584704,
   256: 12584704,
   512: 12584704,
   1024: 12584704},
  'mem_acc': {1: 539346304,
   2: 1066107904,
   4: 2119631104,
   8: 4226677504,
   16: 8440770304,
   32: 16868955904,
   64: 33725327104,
   128: 67438069504,
   256: 134863554304,
   512: 269714523904,
   1024: 539416463104},
  'param_frac': {1: 0.02333325343414238,
   2: 0.011804343587344794,
   4: 0.005937214252164513,
   8: 0.002977445993476014,
   16: 0.0014909425972693784,
   32: 0.0007460274406797098,
   64: 0.00037315291149562753,
   128: 0.000186611273017736,
   256: 9.331434326306157e-05,
   512: 4.665934862476778e-05,
   1024: 2.3330218598785437e-05},
  'ai': {1: 16.38994303741442,
   2: 16.58341555640507,
   4: 16.681874847596124,
   8: 16.731544229024767,
   16: 16.756490001033914,
   32: 16.768990802384163,
   64: 16.775248200124913,
   128: 16.778378650546728,
   256: 16.779944313931523,
   512: 16.78072725520317,
   1024: 16.78111875323828}},
 'lstm': {'flops': {1: 129003520,
   2: 249520128,
   4: 490553344,
   8: 972619776,
   16: 1936752640,
   32: 3865018368,
   64: 7721549824,
   128: 15434612736,
   256: 30860738560,
   512: 61712990208,
   1024: 123417493504},
  'param_mem': {1: 30129152,
   2: 30129152,
   4: 30129152,
   8: 30129152,
   16: 30129152,
   32: 30129152,
   64: 30129152,
   128: 30129152,
   256: 30129152,
   512: 30129152,
   1024: 30129152},
  'mem_acc': {1: 35322336,
   2: 36284352,
   4: 38208384,
   8: 42056448,
   16: 49752576,
   32: 65144832,
   64: 95929344,
   128: 157498368,
   256: 280636416,
   512: 526912512,
   1024: 1019464704},
  'param_frac': {1: 0.8529773342284044,
   2: 0.830362135170555,
   4: 0.7885481887954225,
   8: 0.7163979230961207,
   16: 0.6055797392279748,
   32: 0.4624948913829419,
   64: 0.31407649363264695,
   128: 0.1912981853881813,
   256: 0.10736009399435888,
   512: 0.057180559037474515,
   1024: 0.029553894197400284},
  'ai': {1: 3.6521797425855413,
   2: 6.876797138336658,
   4: 12.838892741446484,
   8: 23.1265316557404,
   16: 38.927685674004096,
   32: 59.329623691408095,
   64: 80.49205281754038,
   128: 97.99855663266301,
   256: 109.96697791351497,
   512: 117.12189178001528,
   1024: 121.06107550340458}},
 'bert-large': {'flops': {1: 79136298142,
   2: 158272596284,
   4: 316545192568,
   8: 633090385136,
   16: 1266180770272,
   32: 2532361540544,
   64: 5064723081088,
   128: 10129446162176,
   256: 20258892324352,
   512: 40517784648704,
   1024: 81035569297408},
  'param_mem': {1: 1213726720,
   2: 1213726720,
   4: 1213726720,
   8: 1213726720,
   16: 1213726720,
   32: 1213726720,
   64: 1213726720,
   128: 1213726720,
   256: 1213726720,
   512: 1213726720,
   1024: 1213726720},
  'mem_acc': {1: 2601011072,
   2: 3988295424,
   4: 6762864128,
   8: 12312001536,
   16: 23410276352,
   32: 45606825984,
   64: 89999925248,
   128: 178786123776,
   256: 356358520832,
   512: 711503314944,
   1024: 1421792903168},
  'param_frac': {1: 0.46663650649773164,
   2: 0.30432217049325583,
   4: 0.17946933385440328,
   8: 0.09858078042397021,
   16: 0.0518458945870713,
   32: 0.026612830290487773,
   64: 0.013485863645503103,
   128: 0.006788707615366562,
   256: 0.0034059146871703223,
   512: 0.001705862354408747,
   1024: 0.0008536592898273774},
  'ai': {1: 30.425206179975845,
   2: 39.68427096237092,
   4: 46.806380636485265,
   8: 51.42059016845139,
   16: 54.08653666592991,
   32: 55.52593248722055,
   64: 56.27474764153262,
   128: 56.65678044939952,
   256: 56.84974861005992,
   512: 56.94672645607142,
   1024: 56.9953395581359}}}

In [6]:
const.keys() == models_metrics.keys()

True

In [7]:
models_metrics

{'lstm': {}, 'yolov8n': {}, 'mobnet': {}, 'resnet50': {}, 'bert-large': {}}

In [8]:
for model_name in models_metrics:
    df = filtered_df_dict[model_name]
    for bs in df['bs']:
        models_metrics[model_name][bs] = {}
        models_metrics[model_name][bs]['ai'] = const[model_name]['flops'][bs] / const[model_name]['mem_acc'][bs]
        models_metrics[model_name][bs]['perf'] = const[model_name]['flops'][bs] / df[df['bs'] == bs]['observed_time'].item() * 1000

In [11]:
filtered_df_dict['yolov8n'][filtered_df_dict['yolov8n']['bs'] == 16]['observed_time'].item()

107.45811462402344

In [12]:
df = filtered_df_dict['lstm']
# df[df[model_name]['bs'] == bs]['observed_time'].item()
observed_time_bs_16 = df[df['bs'] == 16]['observed_time'].item()
observed_time_bs_16

1.8838880062103271

In [13]:
models_metrics_df = pd.DataFrame.from_dict({(i, j): models_metrics[i][j] 
                                            for i in models_metrics.keys() 
                                            for j in models_metrics[i].keys()},
                                           orient='index')
models_metrics_df.to_csv('runtime_data/batch_size-performance.csv')